In [3]:
# IMPORT STATEMENTS

import sqlalchemy
# import pymysql
import os
import shutil
import model_config as mcfig
import pandas as pd
import numpy as np
import datetime as dt

import sys
sys.path.append('../')
import private_info.db_info as dbi

In [4]:
#CUSTOM ERROR CLASSES

In [5]:
class AssetNotExistInDB(Exception):
    """One of the assets requested does not exist in the database!"""
    pass

In [6]:
#GET NECESSARY VARIABLES FROM MODEL CONFIGURATION FILE

In [7]:
asset_pred_list = mcfig.asset_pred_list

In [8]:
train_start = mcfig.train_start
train_end = mcfig.train_end

In [9]:
train_start_dt = dt.datetime.strptime(train_start, '%Y-%m-%d')
train_end_dt = dt.datetime.strptime(train_end, '%Y-%m-%d')

In [10]:
test_start = mcfig.test_start
test_end = mcfig.test_end

In [11]:
test_start_dt = dt.datetime.strptime(test_start, '%Y-%m-%d')
test_end_dt = dt.datetime.strptime(test_end, '%Y-%m-%d')

In [12]:
# DELETE OLD FILES

In [13]:
if os.path.isdir('p_data') == True:
    shutil.rmtree('p_data')
    
os.mkdir('p_data')


In [14]:
engine = sqlalchemy.create_engine(f'mysql+pymysql://{dbi.username}:{dbi.password}@{dbi.hostname}:{dbi.port}/{dbi.database}')

In [15]:
db_sqla_conn = engine.connect()

In [16]:
# GET ALL DATABASE TABLES

In [17]:
db_tables = engine.execute(" SHOW TABLES; ")
db_tables = db_tables.fetchall()

In [18]:
db_tables_list = []
for each in db_tables:
    db_tables_list.append(each[0])

In [19]:
# BREAK TABLES INTO MACRO AND ASSET LISTS

In [20]:
macro_list = ['fvx', 'tnx', 'vix']

In [21]:
asset_list = []

for each in db_tables_list:
    if each not in macro_list:
        asset_list.append(each)

In [22]:
# COMPARE USER INPUT TO EXISTING DB TABLES

In [23]:
for i in range (0, len(asset_pred_list)):
    asset_pred_list[i] = asset_pred_list[i].lower()

for each in asset_pred_list:
    if each not in asset_list:
        raise AssetNotExistInDB

In [24]:
# GET MACRO (MINUS FF) INFO FROM DB

In [25]:
tmp_macro_list = []
for each in macro_list:
    tmp_macro_list.append(pd.read_sql(each, engine))
        

In [26]:
# CLEAN AND MERGE MACRO DATA

In [27]:
inc = 0
for each in tmp_macro_list:
    each.drop(['index', 'High', 'Low', 'Open', 'Volume', 'Adj Close' ], axis=1, inplace=True)
    each.rename({'Close': macro_list[inc] + '_close' }, axis=1, inplace=True)
    inc = inc + 1

In [28]:
macro_merged = tmp_macro_list[0]
for i in range(0, len(tmp_macro_list) - 1):
    macro_merged = pd.merge(macro_merged, tmp_macro_list[i + 1], on='Date')

In [29]:
# GET FED FUNDS DATA

In [30]:
ff_df = pd.read_sql('fed_funds', engine)

In [31]:
ff_df.drop('index', axis=1, inplace=True)

In [32]:
# MERGE IN FED FUNDS DATA TO MACRO DATA

In [33]:
macro_merged = pd.merge(macro_merged, ff_df, on = 'Date')

In [34]:
macro_merged

,Date,fvx_close,tnx_close,vix_close,DFF
0,2000-06-01,6.435,6.195,22.360001,6.65
1,2000-06-02,6.368,6.148,21.480000,6.44
2,2000-06-05,6.323,6.101,22.709999,6.51
3,2000-06-06,6.345,6.127,23.049999,6.47
4,2000-06-07,6.338,6.114,22.480000,6.50
...,...,...,...,...,...
5418,2021-12-22,1.222,1.457,18.629999,0.08
5419,2021-12-23,1.243,1.493,17.959999,0.08
5420,2021-12-27,1.252,1.481,17.680000,0.08
5421,2021-12-28,1.245,1.481,17.540001,0.08


In [35]:
# GETTING ASSET INFO FROM DB

In [36]:
asset_pred_df_list = []
for each in asset_pred_list:
    asset_pred_df_list.append(pd.read_sql(each, engine))
    

In [37]:
# CLEAN AND MERGE ASSET DATA

In [38]:
inc = 0
for each in asset_pred_df_list:
    each.drop(['index', 'High', 'Low', 'Open', 'Adj Close' ], axis=1, inplace=True)
    each.rename({'Close': asset_pred_list[inc] + '_close' }, axis=1, inplace=True)
    each.rename({'Volume': asset_pred_list[inc] + '_volume' }, axis=1, inplace=True)
    inc = inc + 1

In [39]:
asset_pred_df_list[0]

,Date,spy_close,spy_volume
0,2000-06-01,145.312500,8961600.0
1,2000-06-02,147.843750,8962200.0
2,2000-06-05,147.125000,6998100.0
3,2000-06-06,146.468750,4858900.0
4,2000-06-07,147.484375,4919500.0
...,...,...,...
5418,2021-12-22,467.690002,58890200.0
5419,2021-12-23,470.600006,56384300.0
5420,2021-12-27,477.260010,56808600.0
5421,2021-12-28,476.869995,38944403.0


In [40]:
# MERGING ASSET DATA WITH MACRO DATA

In [41]:
base_data_list = []

for each in asset_pred_df_list:
    base_merged = pd.merge(macro_merged, each, on = 'Date')
    base_data_list.append(base_merged)

In [42]:
base_data_list[0]

,Date,fvx_close,tnx_close,vix_close,DFF,spy_close,spy_volume
0,2000-06-01,6.435,6.195,22.360001,6.65,145.312500,8961600.0
1,2000-06-02,6.368,6.148,21.480000,6.44,147.843750,8962200.0
2,2000-06-05,6.323,6.101,22.709999,6.51,147.125000,6998100.0
3,2000-06-06,6.345,6.127,23.049999,6.47,146.468750,4858900.0
4,2000-06-07,6.338,6.114,22.480000,6.50,147.484375,4919500.0
...,...,...,...,...,...,...,...
5418,2021-12-22,1.222,1.457,18.629999,0.08,467.690002,58890200.0
5419,2021-12-23,1.243,1.493,17.959999,0.08,470.600006,56384300.0
5420,2021-12-27,1.252,1.481,17.680000,0.08,477.260010,56808600.0
5421,2021-12-28,1.245,1.481,17.540001,0.08,476.869995,38944403.0


In [43]:
# FIXING FED FUNDS COLUMN
# Idea is to shift it down one row so yesterdays value is used to help with current day's prediciton. 
# This seems to make logical sense because the fed funds rate for the current day is not posted until 9 am the day of.
# In theory this could me that the previous days value will affect current day the most. 
# We will operate under this assumption

In [44]:
for each in base_data_list:
    each['DFF'] = each['DFF'].shift(1)

In [45]:
base_data_list[0]

,Date,fvx_close,tnx_close,vix_close,DFF,spy_close,spy_volume
0,2000-06-01,6.435,6.195,22.360001,NaN,145.312500,8961600.0
1,2000-06-02,6.368,6.148,21.480000,6.65,147.843750,8962200.0
2,2000-06-05,6.323,6.101,22.709999,6.44,147.125000,6998100.0
3,2000-06-06,6.345,6.127,23.049999,6.51,146.468750,4858900.0
4,2000-06-07,6.338,6.114,22.480000,6.47,147.484375,4919500.0
...,...,...,...,...,...,...,...
5418,2021-12-22,1.222,1.457,18.629999,0.08,467.690002,58890200.0
5419,2021-12-23,1.243,1.493,17.959999,0.08,470.600006,56384300.0
5420,2021-12-27,1.252,1.481,17.680000,0.08,477.260010,56808600.0
5421,2021-12-28,1.245,1.481,17.540001,0.08,476.869995,38944403.0


In [46]:
# PROCESSING BASE DATA

In [47]:
p_data_list = base_data_list

In [48]:
#CREATING DELTA PRICE COLUMN

In [49]:
for each in p_data_list:
    
    each[ each.columns[5] + "_next" ] = each[each.columns[5]].shift(-1)
    each ['DeltaPrice'] = each[ each.columns[5] + "_next" ] - each[ each.columns[5] ]
    each.drop( each.columns[5] + "_next", axis=1, inplace=True )
    


In [50]:
#CREATING ZSCORE COLUMN

In [51]:
for each in p_data_list:
    
    closePriceList = each[each.columns[5]].tolist()
    std_dev_14 = []
    
    for i in range(0, len(closePriceList)):
        if (0 <= i <= 13):
            std_dev_14.append(None)
            continue
        temp_list = []
        for j in range(i - 14, i):
            temp_list.append(closePriceList[j])
            
        mean = np.mean(temp_list)
        std_dev = np.std(temp_list)
        obs_val = closePriceList[i]
        z = (obs_val - mean)/std_dev
        
        std_dev_14.append(z)
            
    each['Zscore'] = std_dev_14

In [52]:
# CREATING 100 SMA COLUMN

In [53]:
for each in p_data_list:
    
    sma100 = []
    for i in range(0, len(closePriceList)):
        if (0 <= i <= 99):
            sma100.append(None)
            continue
        temp_list = []
        for j in range(i - 100, i):
            temp_list.append(closePriceList[j])
            
        mean = np.mean(temp_list)
        
        sma100.append(mean)
            
    each['SMA100'] = sma100

In [54]:
# CREATING 200 SMA COLUMN

In [55]:
for each in p_data_list:
    
    sma200 = []
    for i in range(0, len(closePriceList)):
        if (0 <= i <= 199):
            sma200.append(None)
            continue
        temp_list = []
        for j in range(i - 200, i):
            temp_list.append(closePriceList[j])
            
        mean = np.mean(temp_list)
        
        sma200.append(mean)
            
    each['SMA200'] = sma200
    

In [56]:
# DROPPING FIRST 200 ROW
# Gets rid of all NaNs

In [57]:
for each in p_data_list:
    
    each.drop(each.head(200).index, axis=0, inplace=True)
    each.reset_index(drop=True, inplace=True)
   

In [58]:
# CONVERTING DATE COLUMN TO DATETIME TYPE
# This will allow us to more easily query by date later on

In [59]:
for each in p_data_list:
    for i in range (0, each.shape[0]):
        value = each.loc[i, 'Date']
        dt_value = dt.datetime.strptime(value, '%Y-%m-%d')
        each.loc[i, 'Date'] = dt_value
        

In [60]:
p_data_list[1]

,Date,fvx_close,tnx_close,vix_close,DFF,qqq_close,qqq_volume,DeltaPrice,Zscore,SMA100,SMA200
0,2001-03-19 00:00:00,4.530,4.815,29.780001,5.40,43.259998,53248200.0,-2.910000,-1.018243,63.254150,77.781684
1,2001-03-20 00:00:00,4.493,4.791,30.959999,5.38,40.349998,104796200.0,0.000000,-1.927228,62.824875,77.561109
2,2001-03-21 00:00:00,4.441,4.768,31.930000,5.14,40.349998,96200700.0,2.450001,-1.590109,62.403375,77.294734
3,2001-03-22 00:00:00,4.382,4.708,32.840000,5.05,42.799999,130208100.0,0.000000,-0.578583,62.028125,77.027188
4,2001-03-23 00:00:00,4.480,4.799,30.450001,5.04,42.799999,95277500.0,-0.989998,-0.493341,61.657375,76.784938
...,...,...,...,...,...,...,...,...,...,...,...
5218,2021-12-22 00:00:00,1.222,1.457,18.629999,0.08,393.950012,37042200.0,2.970001,0.587962,378.846901,358.603701
5219,2021-12-23 00:00:00,1.243,1.493,17.959999,0.08,396.920013,29574300.0,6.559998,1.039448,379.140401,359.019051
5220,2021-12-27 00:00:00,1.252,1.481,17.680000,0.08,403.480011,32820700.0,-1.870026,2.069535,379.441501,359.413451
5221,2021-12-28 00:00:00,1.245,1.481,17.540001,0.08,401.609985,30843855.0,-1.089996,1.393975,379.802901,359.853551


In [61]:
# CREATING TRAINING AND TEST SETS

In [62]:
train_list = []
test_list = []

for each in p_data_list:
    
    
    
    mask_train = (each['Date'] >= train_start_dt) & (each['Date'] <= train_end_dt)
    tmp = each.loc[mask_train]
    tmp.reset_index(drop=True, inplace=True)
    train_list.append(tmp)
    
    mask_test = (each['Date'] >= test_start_dt) & (each['Date'] <= test_end_dt)
    tmp = each.loc[mask_test]
    tmp.reset_index(drop=True, inplace=True)
    test_list.append(tmp)
    

In [63]:
inc = 0
for each in train_list:
    each.to_csv(f'./p_data/{asset_pred_list [inc]}_train.csv')
    inc = inc + 1

In [64]:
inc = 0
for each in test_list:
    each.to_csv(f'./p_data/{asset_pred_list [inc]}_test.csv')
    inc = inc + 1

In [65]:
test_list[0]

,Date,fvx_close,tnx_close,vix_close,DFF,spy_close,spy_volume,DeltaPrice,Zscore,SMA100,SMA200
0,2018-01-02 00:00:00,2.250,2.465,9.770000,1.33,268.769989,86655700.0,1.700012,2.723571,149.473100,143.749900
1,2018-01-03 00:00:00,2.247,2.447,9.150000,1.42,270.470001,90070400.0,1.139984,4.419999,149.616800,143.883900
2,2018-01-04 00:00:00,2.268,2.453,9.220000,1.42,271.609985,80636400.0,1.810028,3.752096,149.806800,144.025000
3,2018-01-05 00:00:00,2.285,2.476,9.220000,1.42,273.420013,83524000.0,0.500000,3.715703,149.988900,144.177500
4,2018-01-08 00:00:00,2.285,2.480,9.520000,1.42,273.920013,57319200.0,0.619995,2.822429,150.168800,144.333650
...,...,...,...,...,...,...,...,...,...,...,...
961,2021-10-26 00:00:00,1.183,1.619,15.980000,0.08,455.959991,56075100.0,-2.019989,1.527512,363.326501,344.567051
962,2021-10-27 00:00:00,1.138,1.529,16.980000,0.08,453.940002,72438000.0,4.380005,1.060143,363.761701,344.890551
963,2021-10-28 00:00:00,1.188,1.568,16.530001,0.08,458.320007,51437900.0,0.929993,1.465985,364.195701,345.220951
964,2021-10-29 00:00:00,1.188,1.557,16.260000,0.08,459.250000,70108200.0,0.790009,1.375413,364.670301,345.561851
